<a href="https://colab.research.google.com/github/YellaCRE/Article_rec/blob/main/Article_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 토크나이저

In [ ]:
EMAIL_PATTERN = re.compile("(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", re.UNICODE)
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.UNICODE)
pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)', re.UNICODE) # 본문 시작 전 꺽쇠로 쌓인 바이라인 제거
TEST = re.compile('-|“|…|,|”|≪|≫|!', re.UNICODE)

In [ ]:
tokenizer = Mecab()
sentences = []

In [ ]:
for content in data['title'] :
  #content = re.sub(EMAIL_PATTERN, ' ',content)
  #content = re.sub(URL_PATTERN, ' ',content)
  #content = re.sub(pattern_bracket, ' ',content)
  #content = re.sub(EXCEPT, ' ',content)
  tokens = tokenizer.morphs(content)
  sentences.append(tokens)

보유데이터로 학습

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(sentences)

#단어벡터평균

In [ ]:
def get_document_vectors(document_list, model):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in model.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = model[word]
                else:
                    doc2vec = doc2vec + model[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [ ]:
target_output = []
compared_output = []
for i in range(len(test['sentence1'])):
  target = get_document_vectors(test['sentence1'][i:i+1],model)
  compared = get_document_vectors(test['sentence2'][i:i+1],model)

  if target== [] or compared == []:
    continue
  else:
    target_output.append(target)
    compared_output.append(compared)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [ ]:
print('target_output 문서 벡터의 수 :',len(target_output))
print('compared_output 문서 벡터의 수 :',len(compared_output))

target_output 문서 벡터의 수 : 460
compared_output 문서 벡터의 수 : 460


#코사인유사도

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import random

In [ ]:
cosim=[]
for i in range(0,len(target_output)):
  cosim.append(cosine_similarity(target_output[i],compared_output[i])[0][0])

In [ ]:
y_pred=round(pd.Series(cosim)*5,1)
y_test=test['score'][:len(target_output)]
y_rand = [random.randint(0,5) for value in range(0,len(target_output))]

In [ ]:
test['pred_score'] = y_pred
test[['pred_score','score','sentence1','sentence2']].head(10)

,pred_score,score,sentence1,sentence2
0,4.4,1.40,비벤디 주식은 월요일에 3.6퍼센트 하락한 후 파리에서 15.80유로로 1.9% 하...,"뉴욕에서는, 비벤디 주식은 18.29달러로 1.4% 하락했다."
1,2.2,3.60,브레머는 한 가지 계획이 동네를 청소하고 지역 사회 프로젝트를 구축하기 위해 앞으로...,브레머는 이라크 전역의 이웃들을 청소하고 지역 사회 프로젝트를 건설하기 위해 앞으로...
2,1.3,3.50,그레이 데이비스 주지사는 어제 화재가 거의 20억 달러의 비용이 들 수 있다고 추정했다.,주 관리들은 그 비용을 거의 20억 달러로 추정했다.
3,3.9,3.20,American Express 회사는 필요한 500만 달러의 금액 보다 최소 300...,"시는 연방 기금을 요청했지만, American Express가 최소 300만 달러를..."
4,1.0,4.00,대서양 폭풍 시즌의 첫 번째 허리케인인 클로데트는 7월 15일 85mph의 바람으로...,대서양 폭풍 시즌의 첫 번째 허리케인인 클로데트는 7월 15일 텍사스 중부 해안을 ...
5,3.9,2.00,미국 2위 청량음료 제조사 펩시코 주식회사의 결과(nyse: pep- news-pe...,월스트리트도 알루미늄 제조사인 알코아 주식회사가( nyse : pep - news ...
6,0.2,3.60,테일러의 보좌관들은 갑작스러운 출발이 더 많은 혼란을 유발할 수 있다고 경고했다. ...,"그의 보좌관들은 갑작스러운 출발이 많은 유혈사태를 촉발시킬 수 있다고 경고했는데, ..."
7,2.6,4.20,그러나 오데뜨가 카리브해에서 처음으로 12월에 형성되었다고 센터는 말했다.,12월에 카리브해에서 처음으로 개발된 이름있는 폭풍이다.
8,2.3,4.00,AFL-CIO는 10월까지 후보자를 지지할지 결정하기 위해 기다리고 있다.,AFL-CIO는 수요일에 경선 전에 후보를 지지할 것인지 여부를 10월에 결정할 것...
9,4.4,3.25,"그것은 학계, 산업계, 정부, 그리고 대중의 300명 이상의 지도자들로부터 협의를 ...",NIH 로드맵이라고 불리는 이 계획은 산업계와 학계에서 300명 이상의 컨설턴트의 ...


#결과지표

In [ ]:
mean_absolute_error(y_test, y_pred)

1.695076093778662

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

2.059009831374267

마이너스값을 모두 0으로 치환

In [ ]:
y_pred_zero = np.where(y_pred < 0, 0, y_pred)

In [ ]:
mean_absolute_error(y_test, y_pred_zero)

1.6783369632888099

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred_zero))

2.0304012685743933

랜덤값과 비교

In [ ]:
mean_absolute_error(y_test, y_rand)

1.7099543478260868

In [ ]:
np.sqrt(mean_squared_error(y_test, y_rand))

2.089532126223892

In [ ]:
pd.DataFrame(cosim).describe()

,0
count,460.000000
mean,0.562896
std,0.316653
min,-0.209214
25%,0.301550
50%,0.596169
75%,0.824175
max,1.000000
